# RNN Sequence Prediction
Try to guess the key

In [56]:
import pickle
import os
import random
import tensorflow as tf
from tensorflow.contrib import rnn
import matplotlib.pyplot as plt
import numpy as np
import time

%matplotlib inline

# TF settings
logs_path = os.getcwd()
writer = tf.summary.FileWriter(logs_path)

In [62]:
# Load data
# 'sbox', ' x_train', 'y_train'
with open("training_data.pkl", 'rb') as f:
    data = pickle.load(f)

In [63]:
# one hot encoding
def encode(label_data):
    # get max 
    max_num = np.amax(label_data)
    output = np.zeros([len(label_data), len(label_data[0]), max_num + 1])
    for i in range(len(label_data)):
        temp = np.zeros((len(label_data[0]), max_num + 1))
        temp[np.arange(len(label_data[0])), label_data[i]] = 1
        output[i] = temp
    return output

data['y_train'] = encode(data['y_train'])
print(data['y_train'])
print(data['y_train'].shape)

[[[1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[1. 0. 0. 0.]
  [1. 0. 0. 0.]]

 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]]

 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]]

 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]]

 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]]

 [[1. 0. 0. 0.]
  [0. 0. 1. 0.]]

 [[1. 0. 0. 0.]
  [0. 0. 1. 0.]]

 [[1. 0. 0. 0.]
  [0. 0. 1. 0.]]

 [[1. 0. 0. 0.]
  [0. 0. 1. 0.]]

 [[1. 0. 0. 0.]
  [0. 0. 0. 1.]]

 [[1. 0. 0. 0.]
  [0. 0. 0. 1.]]

 [[1. 0. 0. 0.]
  [0. 0. 0. 1.]]

 [[1. 0. 0. 0.]
  [0. 0. 0. 1.]]

 [[0. 1. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 1. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 1. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 1. 0. 0.]
  [1. 0. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 1. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 1. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 1. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 1. 0. 0.]]

 [[0. 1. 0. 0.]
  [0. 0. 1. 0.]]

 [[0. 1. 0. 0.]
  [0. 0. 1. 0.]]

 [[0. 1. 0. 0.]
  [0. 0. 1. 0.]]

 [[0. 1. 0. 0.]
  [0. 0. 1. 0.]]

 [[0. 1. 0. 0.]
  [0. 0. 0. 1.]]

 [[0. 1. 0. 0.

In [64]:
# Clear graph
tf.reset_default_graph()

In [65]:
# Param
# Training
learning_rate = 0.001
training_steps = 10000
batch_size = 20
display_step = 200

# Network
n_input = 2
n_hidden = 512
n_output = 2
n_sequence = 1

# num classes this is proportional to 2**bit_size
n_classes = 2**2

# tf graph input
x = tf.placeholder("float", [None, n_input, n_sequence])
y = tf.placeholder("float", [None, n_classes, n_output])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_output]))
}

biases = {
    'out': tf.Variable(tf.random_normal([n_output], mean=1.0))
}


In [66]:
# RNN Model
def RNN(x, weights, biases):
    # Reshape input to [1, n_input]
    x = tf.reshape(x, [-1, n_input * n_sequence])

    # generate a n_input-element sequence of inputs
    x = tf.split(x, n_input, 1)

    # 1-layer LSTM with n_hidden units
    rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # 2-layer LSTM cell
    # rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden), rnn.BasicLSTMCell(n_hidden)])

    # Get lstm output
    _outputs, _states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # Linear Activation, using rnn inner loop last output
    # Return last two outputs
    pred = tf.matmul(_outputs[-1], weights['out']) + biases['out']
    
    return pred

pred = RNN(x, weights, biases)

In [67]:
# Loss and optimizer

# Cross entropy loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

# Simple square loss
#cost = tf.reduce_mean(tf.square(pred - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Model eval
print(tf.shape(pred))
print(tf.shape(y))
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize variables
init = tf.global_variables_initializer()

Tensor("Shape:0", shape=(2,), dtype=int32)
Tensor("Shape_1:0", shape=(3,), dtype=int32)


In [68]:
# Launch the graph
start = time.time()
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0
    
    writer.add_graph(session.graph)
    
    while step < training_steps:
       # generate a minibatch. Add some randomness on selection
        
        symbols_in_keys = np.reshape(data['x_train'], [-1, 2, 1])
        
        symbols_out_onehot = np.reshape(np.zeros([n_classes, n_output], dtype=float), [-1, 4, 2])
        
        _, acc, loss, onehot_pred = session.run(
            [optimizer, accuracy, cost, pred], 
            feed_dict={x: symbols_in_keys, y: symbols_out_onehot}) 
        
        loss_total += loss
        acc_total += acc
        
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                 "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                 "{:.2f}%".format(100*acc_total/display_step))
            
            acc_total = 0
            loss_total = 0
            data_in = [data['x_train'][i] for i in range(len(data['x_train']))]
            data_out = data['y_train']
            data_out_pred = onehot_pred
            #symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("{} - [{}] vs [{}]".format(data_in, data_out, data_out_pred))
            
        step += 1
        
#     print("Optimization Finished")
#     print("Elapsed time: ", time.time() - start_time)
#     print("Run on command line.")
#     print("\ttensorboard --logdir=%s" & (logs_path))
#     print("Point your web browser to: http://localhost:6006/")
    
#     while True:
#         prompt = "%s input: " % n_input
#         sentence = input(prompt)
#         sentence = sentence.strip()
#         words = sentence.split(' ')
        
#         if len(n_classes) != n_input:
#             continue
        
#         try:
#             symbols_in_keys = [i for i in range(n_classes)]
            
#             for i in range(32):
#                 keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
#                 onehot_pred = session.run(pred, feed_dict={x: keys})
#                 onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
#                 sentence = "%s %s" % (setence, onehot_pred_index)
#                 symbols_in_keys = symbols_in_keys[1:]
#                 symbols_in_keys.append(onthot_pred_index)
#             print(sentence)
            
#         except:
#             print("word not in dict")

InvalidArgumentError: logits and labels must be broadcastable: logits_size=[64,2] labels_size=[4,2]
	 [[Node: softmax_cross_entropy_with_logits_sg = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](softmax_cross_entropy_with_logits_sg/Reshape, softmax_cross_entropy_with_logits_sg/Reshape_1)]]

Caused by op 'softmax_cross_entropy_with_logits_sg', defined at:
  File "C:\Users\sjoshia\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\sjoshia\AppData\Local\Continuum\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\sjoshia\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\sjoshia\AppData\Local\Continuum\anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\sjoshia\A